# CS202 G3T2 Reading Assignment
In the following code, we intend to experiment and test different algorithms and possible solutions to the Travelling Salesman Problem (TSP).

## Table of contents
1. [Travelling Salesman Problem (TSP)](#the-travelling-salesman-problem-tsp)
1. [Algorithms](#list-of-algorithms)
1. [Experiment](#experimental-data)
1. [Results](#results)
1. [Conclusion](#conclusion)



In [193]:
# Import statements
import time
import random
from tabulate import tabulate
from functools import cmp_to_key

## The Travelling Salesman Problem (TSP)

Firstly, we define TSP as a problem involving an N x N adjacency matrix which represent the edge weights. For demonstration purposes, we chose to use a complete graph to represent it, with the assumptions that there are edges between every two nodes and that all edge weights are positive.

The following piece of code was taken and altered from the Wk12 exercises.

The function generates a random undirected graph with num_vertices number of "cities", with additional cost_min and cost_limit parameters to dictate the range of edge weights possible.

In [194]:
def generate_graph(
    num_vertices: int, cost_min: int, cost_limit: int
) -> list[list[int]]:

    graph: list[list[int]] = [
        [0 for _ in range(num_vertices)] for _ in range(num_vertices)
    ]

    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            weight: int = random.randint(cost_min, cost_limit)
            graph[i][j] = weight
            graph[j][i] = weight

    return graph

## Helper / utility functions

get_cost is a function used by several of the algorithms to obtain the cost of an entire path. It assumes that the path visits all nodes, while starting and ending at index 0.

apply_2_opt is a helper function for the local search functions which does 2-opt swapping by a pair of edges at x to x + 1 and y to y + 1.

In [195]:
# Must be of length N
# Will auto include the path back to start
def get_cost(graph: list[list[int]], visited: list[int]) -> int:
    n: int = len(visited)
    current_cost: int = 0

    for i in range(1, n):
        current_cost += graph[visited[i - 1]][visited[i]]

    current_cost += graph[visited[-1]][0]
    return current_cost


# A, B, C -> A, rev(B), C
# x and y must be at least 2 apart
# If 1 apart, means adjacent, no diff
def apply_2_opt(visited: list[int], x: int, y: int) -> list[int]:
    new_path: list[int] = visited.copy()
    new_path[x + 1 : y + 1] = reversed(visited[x + 1 : y + 1])

    return new_path


# Generates a random path for the graph
def generate_random_path(graph: list[list[int]]) -> tuple[int, list[int]]:
    n: int = len(graph)
    path: list[int] = [0]
    cost: int = 0
    while len(path) != n:
        index: int = random.randint(1, n - 1)
        while index in path:
            index = random.randint(1, n - 1)
        cost += graph[path[-1]][index]
        path.append(index)
    cost += graph[path[-1]][index]
    return cost, path

## Perturbation operators
The following code shows the operators used by ILS to escape the locality.

In [196]:
# A, B, C, D -> A, D, C, B
# Requires 4 distinct values
# Removes 4 edges, introduce 4 edges
def double_bridge_move(
    path: list[int],
) -> list[int]:

    n: int = len(path)

    vertices: list[int] = []
    while True:
        while len(vertices) != 4:
            index = random.randint(0, n - 1)
            while index in vertices:
                index = random.randint(0, n - 1)
            vertices.append(index)

        vertices.sort()
        continue_loop = False
        for i in range(1, 4):
            if vertices[i] - vertices[i - 1] == 1:
                continue_loop = True
                break
        if continue_loop:
            vertices.clear()
            continue

        break
    for i in range(4):
        vertices[i] += 1

    new_path: list[int] = path.copy()
    new_path[vertices[1] : vertices[2]], new_path[vertices[3] :] = (
        new_path[vertices[3] :],
        new_path[vertices[1] : vertices[2]],
    )

    return new_path


# Randomises vertices in a segment of the original path
def randomise_segments(path: list[int]) -> list[int]:
    n: int = len(path)
    chosen_vertices: list[int] = []

    # Get random segment index
    while True:
        # Get 2 unique index
        for _ in range(2):
            index: int = random.randint(1, n - 1)
            while index in chosen_vertices:
                index = random.randint(1, n - 1)
            chosen_vertices.append(index)

        chosen_vertices.sort()

        # NOTE: It is possible to add some criteria
        # to the chosen_vertices here, such as
        # setting a minimum length for the slices
        if chosen_vertices[1] - chosen_vertices[0] <= 2:
            chosen_vertices.clear()
            # Randomise again
            continue

        break

    new_path: list[int] = path.copy()
    segment: list[int] = new_path[chosen_vertices[0] : chosen_vertices[1]]
    random.shuffle(segment)
    new_path[chosen_vertices[0] : chosen_vertices[1]] = segment

    return new_path


# Swap random pairs of vertices
def swap_vertices(path: list[int]) -> list[int]:
    n: int = len(path)
    new_path: list[int] = path.copy()

    # WARN: hardcoded for 5 - 10 pairs
    for _ in range(random.randint(5, 10)):
        index1: int = random.randint(1, n - 1)
        index2: int = random.randint(1, n - 1)
        while index2 == index1:
            index2 = random.randint(1, n - 1)

        new_path[index1], new_path[index2] = new_path[index2], new_path[index1]

    return new_path

## List of algorithms
1. [Brute force](#brute-force-algorithm)
1. [Greedy](#greedy-algorithm)
1. [Local search](#local-search)
1. [Iterated local search (ILS) - Double bridge move](#iterated-local-search-ils---double-bridge-move)
1. [Iterated local search (ILS) - Random segments](#iterated-local-search-ils---random-segments)
1. [Iterated local search (ILS) - Swap vertices](#iterated-local-search-ils---swap-vertices)
1. [Iterated local search (ILS) - Random initialisation](#iterated-local-search-ils---random-initialisation)

## Brute force algorithm

The most intuitive algorithm, is of course the brute force algorithm, which involves running through every path possible and then keep track of the path with the minimum cost. This guarantees that the optimal solution is found, but at the cost of exponential time, hence it is often not used.

In [197]:
def brute_force(graph: list[list[int]]) -> tuple[int, int, list[int]]:
    n: int = len(graph)
    optimal_path: list[int] = []
    iteration: int = 0
    min_cost: float = float("inf")

    # Recursive function to search all paths
    # path contains the index of all the vertices visited
    # cost is the current cost of the path
    def aux_func(path: list[int], cost: int) -> None:
        if len(path) == n:
            nonlocal iteration
            iteration += 1

            # Add the last path back to start vertex
            cost += graph[path[-1]][0]
            nonlocal min_cost

            # Compare with current best solution
            if cost < min_cost:
                # Replace with new best solution
                min_cost = cost
                nonlocal optimal_path
                optimal_path = path.copy()
            return

        for i in range(n):
            # Skip if already visited
            if i in path:
                continue

            # path[-1] is the last vertex visited
            # Add the path from last visited to current
            cost += graph[path[-1]][i]
            path.append(i)
            aux_func(path, cost)
            path.pop()
            cost -= graph[path[-1]][i]

    # Always start with the first vertex, with 0 cost
    aux_func([0], 0)
    return min_cost, iteration, optimal_path

## Greedy algorithm

The greedy algorithm does not guarantee an optimal solution. However, it makes up for it by offering a much faster speed. By picking the smallest edge weight for vertices not visited, it is able to iterate through the graph quickly, creating a single path. While not necessarily being the optimal solution, it offers a good initial solution as opposed to forming a completely random path. 

In [198]:
def greedy(graph: list[list[int]]) -> tuple[int, int, list[int]]:
    n: int = len(graph)
    iteration: int = 0

    # Stores the index of all visited vertices
    visited: list[int] = [0]
    cost: int = 0

    # Iterate through remaining n - 1 vertices
    for _ in range(1, n):
        min_cost = float("inf")
        min_idx = -1

        # Gets the last city visited, always valid
        # due to initial list containing the 1st vertex
        last_visited = visited[-1]

        # Iterate through all the vertices
        for j in range(1, n):
            
            # Skip if already visited this vertex
            if j in visited:
                continue

            # Find the next vertex with the smallest
            # edge weight when compared to current vertex
            elif graph[last_visited][j] < min_cost:
                min_cost = graph[last_visited][j]
                min_idx = j

            iteration += 1

        # Update data
        visited.append(min_idx)
        cost += min_cost

    # Add the remaining path back to start vertex
    cost += graph[visited[-1]][0]
    return cost, iteration, visited

## Local search

Local search is used as an approximation algorithm as it too does not guarantee the optimal solution. This is due to its "locality" which does not explore possible "more costly" paths which may eventually lead to a better solution.

Local search requries an initial solution to be used, which is possible a form of optimisation for speed or results.

The following code presents a local search function which utilises the 2-opt helper functions to find a better solution.

Note: the function can also be used with a limit by adding the iteration_limit parameter when called. Default is infinity (no limit).

In [199]:
def local_search_2_opt(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    iteration_limit: float = float("inf"),
) -> tuple[int, int, list[int]]:

    n: int = len(initial_solution)

    # Assumes fully connected graph
    num_edges: int = int((n * (n - 1)) / 2)
    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Terminates only after all edges have been
    # checked and no more better solution
    checked: int = 0
    while checked < num_edges:
        for i in range(n):
            reset_flag = False
            for j in range(i + 1, n):

                # Skip adjacent vertex
                if j - i == 1:
                    checked += 1
                    iteration += 1

                    # Only returns here if there is a
                    # iteration limit specified
                    if iteration >= iteration_limit:
                        return optimal_cost, iteration, optimal_path
                    continue

                new_path: list[int] = apply_2_opt(optimal_path, i, j)
                new_cost: list[int] = get_cost(graph, new_path)

                # If better solution is found,
                # replace current solution
                if new_cost < optimal_cost:
                    optimal_path = new_path
                    optimal_cost = new_cost

                    # Restart the checking variable
                    # as we need to iterate all edges again
                    checked = 0

                    # Need to break out of all the loops
                    # until the while loop to fully reset
                    reset_flag = True
                    iteration += 1
                    break

                checked += 1
                iteration += 1

                # Only returns here if there is a
                # iteration limit specified
                if iteration >= iteration_limit:
                    return optimal_cost, iteration, optimal_path

            if reset_flag:
                break

    return optimal_cost, iteration, optimal_path

## Iterated local search (ILS) - Double bridge move

Basing off the 2-opt local search, we can then create a solution to iterate between solutions generated by the local search. For this perturbation, double bridge move is chosen.

The logic of double bridge move is to remove 4 random edges, and introduce 4 new ones, forming a cross-like shape when bridging the gaps.

Local search is then run on the new paths to try and find better solutions.

Due to the nature of ILS, the termination criterion can also be changed to try and look for less or more solutions.

Below shows 3 variations of this ILS, with a Markovian, random walk, and restart acceptance criteria.

In [200]:
# ILS with 2-opt, double bridge, Markovian
def ILS_2_opt_double_bridge_markovian(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    num_perturbation: int,
) -> tuple[int, int, list[int]]:

    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Start off by searching 2-opt normally
    path: list[int] = optimal_path
    cost: int = initial_cost

    for _ in range(num_perturbation):
        (cost, ls_iteration, path) = local_search_2_opt(graph, cost, path)

        # Only accept better solution
        if cost < optimal_cost:
            optimal_cost = cost
            optimal_path = path
        iteration += ls_iteration

        path = double_bridge_move(path)

    return optimal_cost, iteration, optimal_path

In [201]:
# ILS with 2-opt, double bridge, random_walk
def ILS_2_opt_double_bridge_random_walk(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    num_perturbation: int,
) -> tuple[int, int, list[int]]:

    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Start off by searching 2-opt normally
    path: list[int] = optimal_path
    cost: int = initial_cost

    for _ in range(num_perturbation):
        (cost, ls_iteration, path) = local_search_2_opt(graph, cost, path)
        
        # Accept the solution regardless
        optimal_cost = cost
        optimal_path = path
        iteration += ls_iteration

        path = double_bridge_move(path)

    return optimal_cost, iteration, optimal_path

In [202]:
# ILS with 2-opt, double bridge, restart
def ILS_2_opt_double_bridge_restart(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    num_perturbation: int,
    iteration_limit: int,
) -> tuple[int, int, list[int]]:

    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Start off by searching 2-opt normally
    path: list[int] = optimal_path
    cost: int = initial_cost

    for _ in range(num_perturbation):
        (cost, ls_iteration, path) = local_search_2_opt(
            graph, cost, path, iteration_limit=iteration_limit
        )
        optimal_cost = cost
        optimal_path = path
        iteration += ls_iteration

        # Restart if iteration_limit has been hit
        if cost == iteration_limit:
            cost, path = generate_random_path(graph)
            continue

        path = double_bridge_move(path)

    return optimal_cost, iteration, optimal_path

## Iterated local search (ILS) - Random segments

A variation of ILS, with a different perturbation. This time, we randomly select segments in the graph and then randomly swap the vertices in these segments.

Below shows 3 variations of this ILS, with a Markovian, random walk, and restart acceptance criteria.

In [203]:
# ILS using 2-opt, randomised segments, Markovian
def ILS_2_opt_randomise_segment_markovian(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    num_perturbation: int,
) -> tuple[int, int, list[int]]:

    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Start off by searching 2-opt normally
    path: list[int] = initial_solution
    cost: int = initial_cost

    for _ in range(num_perturbation):

        (cost, ls_iteration, path) = local_search_2_opt(graph, cost, path)

        # Only accept better solution
        if cost < optimal_cost:
            optimal_cost = cost
            optimal_path = path
        iteration += ls_iteration

        path = randomise_segments(path)
        cost = get_cost(graph, path)

    return optimal_cost, iteration, optimal_path

In [204]:
# ILS using 2-opt, randomised segments, random walk
def ILS_2_opt_randomise_segment_random_walk(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    num_perturbation: int,
) -> tuple[int, int, list[int]]:

    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Start off by searching 2-opt normally
    path: list[int] = optimal_path
    cost: int = initial_cost

    for _ in range(num_perturbation):
        (cost, ls_iteration, path) = local_search_2_opt(graph, cost, path)
        
        # Accept solution regardless
        optimal_path = path
        optimal_cost = cost

        iteration += ls_iteration

        path = randomise_segments(path)
        cost = get_cost(graph, path)
        
    return optimal_cost, iteration, optimal_path

In [205]:
# ILS with 2-opt, randomise segments, restart
def ILS_2_opt_randomise_segment_restart(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    num_perturbation: int,
    iteration_limit: int,
) -> tuple[int, int, list[int]]:

    n: int = len(initial_solution)

    # Assumes fully connected graph
    num_edges: int = int((n * (n - 1)) / 2)
    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Start off by searching 2-opt normally
    path: list[int] = optimal_path
    cost: int = initial_cost
    
    for _ in range(num_perturbation):
        (cost, ls_iteration, path) = local_search_2_opt(
            graph, cost, path, iteration_limit=iteration_limit
        )
        # Only accept a better solution
        if cost < optimal_cost:
            optimal_cost = cost
            optimal_path = path
            print("found better sol")
        iteration += ls_iteration

        if cost == iteration_limit:
            cost, path = generate_random_path(graph)
            continue

        path = randomise_segments(path)
        cost = get_cost(graph, path)
        
    return optimal_cost, iteration, optimal_path

## Iterated local search (ILS) - Swap vertices

A variation of ILS, with a different perturbation. This time, we randomly swap vertices around, creating some disruptions which hopefully breaks the locality.

In [206]:
# Basing off 2-opt local search
# Randomly swap a few vertices
def ILS_2_opt_swap_vertices_markovian(
    graph: list[list[int]],
    initial_cost: int,
    initial_solution: list[int],
    num_perturbation: int,
) -> tuple[int, int, list[int]]:

    optimal_path: list[int] = initial_solution
    optimal_cost: int = initial_cost
    iteration: int = 0

    # Start off by searching 2-opt normally
    path: list[int] = optimal_path
    cost: int = initial_cost

    for _ in range(num_perturbation):

        (cost, ls_iteration, path) = local_search_2_opt(graph, cost, path)

        # Only accept better solution
        if cost < optimal_cost:
            optimal_cost = cost
            optimal_path = path

        iteration += ls_iteration

        path = swap_vertices(path)
        cost = get_cost(graph, path)

    return optimal_cost, iteration, optimal_path

## Iterated local search (ILS) - Random initialisation

Another variation of ILS, with a different perturbation. This time, instead of having an initial solution, we generate a random path in each iteration in hopes of finding one that local search may result in the best solution.

In [207]:
# Basing off 2-opt local search
# Randomly initalise paths
# No initial solution required as its all random
def ILS_2_opt_randomise_initial_markovian(
    graph: list[list[int]],
    num_perturbation: int,
) -> tuple[int, int, list[int]]:

    cost, path = generate_random_path(graph)
    tested_paths: list[list[int]] = [path]

    optimal_path: list[int] = path
    optimal_cost: int = cost
    iteration: int = 0

    for _ in range(num_perturbation):
        (cost, ls_iteration, path) = local_search_2_opt(graph, cost, path)

        # Only accept better solutions
        if cost < optimal_cost:
            optimal_cost = cost
            optimal_path = path
        iteration += ls_iteration

        # Generate a completly new path which it has not
        # started with before
        cost, path = generate_random_path(graph)
        while path in tested_paths:
            cost, path = generate_random_path(graph)
        tested_paths.append(path)

    return optimal_cost, iteration, optimal_path

## Experimental Data

Based off the algorithms above, we decided to test the different algorithms with randomly generated "cities", as well as of different sizes. We decided to limit the number of vertices, N, at 200 as it becomes computationally expensive and time consuming to run.

Note: Brute force has been omitted for N > 10 as it simply runs for too long.

In [208]:
# Helper function for pretty printing
def print_data(data: list[tuple[str, int, int, list[int], float]]):
    print(
        tabulate(
            data,
            headers=[
                "Algorithm",
                "Cost",
                "Iterations",
                "Path",
                "Time taken",
            ],
            tablefmt="grid",
        )
    )
    print()


# Comparator function to sort by cost then iteration
def comparator_cost_iteration(algo1, algo2):
    if algo1[1] != algo2[1]:
        return algo1[1] - algo2[1]
    return algo1[2] - algo2[2]


def experiment(N: int, cost_min: int, cost_limit: int):
    print("Experiment starting with following parameters:")
    print("N: %d, cost_min: %d, cost_limit: %d" % (N, cost_min, cost_limit))
    graph: list[list[int]] = generate_graph(N, cost_min, cost_limit)
    if N <= 20:
        print("Graph:")
        for arr in graph:
            print(arr)
        print()

    data: list[tuple[str, int, int, list[int], float]] = []

    if N <= 10:
        start_time: float = time.time()
        (cost, iteration, path) = brute_force(graph)
        elapsed_time: float = time.time() - start_time
        data.append(
            (
                "Brute Force",
                cost,
                iteration,
                path,
                elapsed_time,
            )
        )
        print("Brute Force done.")

    random_cost, random_path = generate_random_path(graph)

    start_time: float = time.time()
    (greedy_cost, iteration, greedy_path) = greedy(graph)
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "Greedy",
            greedy_cost,
            iteration,
            greedy_path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("Greedy done.")

    start_time: float = time.time()
    (cost, iteration, path) = local_search_2_opt(graph, greedy_cost, greedy_path)
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt LS-Greedy",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt LS-Greedy done.")

    start_time: float = time.time()
    (cost, iteration, path) = local_search_2_opt(graph, random_cost, random_path)
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt LS-Random",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt LS-Random done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_double_bridge_markovian(
        graph, greedy_cost, greedy_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Greedy D_Bridge Markovian",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Greedy D_Bridge Markovian done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_double_bridge_random_walk(
        graph, greedy_cost, greedy_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Greedy D_Bridge Random_Walk",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Greedy D_Bridge Random_Walk done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_double_bridge_restart(
        graph, greedy_cost, greedy_path, 10, 500
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Greedy D_Bridge Restart",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Greedy D_Bridge Restart done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_randomise_segment_markovian(
        graph, greedy_cost, greedy_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Greedy R_Segment Markovian",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Greedy R_Segment Markovian done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_swap_vertices_markovian(
        graph, greedy_cost, greedy_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Greedy Swap_Vertices Markovian",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Greedy Swap_Vertices Markovian done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_double_bridge_markovian(
        graph, random_cost, random_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Random D_Bridge Markovian",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Random D_Bridge Markovian done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_double_bridge_random_walk(
        graph, random_cost, random_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Random D_Bridge Random_Walk",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Random D_Bridge Random_Walk done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_double_bridge_restart(
        graph, random_cost, random_path, 10, 500
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Random D_Bridge Restart",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Random D_Bridge Restart done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_randomise_segment_markovian(
        graph, random_cost, random_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Random R_Segment Markovian",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Random R_Segment Markovian done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_swap_vertices_markovian(
        graph, random_cost, random_path, 10
    )
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS-Random Swap_Vertices Markovian",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS-Random Swap_Vertices Markovian done.")

    start_time: float = time.time()
    (cost, iteration, path) = ILS_2_opt_randomise_initial_markovian(graph, 10)
    elapsed_time: float = time.time() - start_time
    data.append(
        (
            "2-opt ILS R_Initial Markovian",
            cost,
            iteration,
            path if N <= 20 else "-",
            elapsed_time,
        )
    )
    print("2-opt ILS R_Initial Markovian done.")

    print()

    # Sort by cost
    data.sort(key=cmp_to_key(comparator_cost_iteration))
    print("SORTED BY COST")
    print_data(data)
    print("WINNER:", data[0][0])
    print()

    # Sort by iteration
    data.sort(key=lambda x: x[2])
    print("SORTED BY ITERATIONS")
    print_data(data)
    print("WINNER:", data[0][0])
    print()


experiment(10, 1, 10)
experiment(20, 5, 20)
experiment(100, 5, 20)
experiment(200, 5, 30)
experiment(500, 10, 50)

Experiment starting with following parameters:
N: 10, cost_min: 1, cost_limit: 10
Graph:
[0, 8, 8, 9, 9, 7, 8, 9, 1, 5]
[8, 0, 5, 10, 6, 8, 9, 9, 1, 3]
[8, 5, 0, 9, 10, 4, 2, 5, 8, 4]
[9, 10, 9, 0, 5, 7, 1, 10, 1, 2]
[9, 6, 10, 5, 0, 6, 6, 5, 8, 8]
[7, 8, 4, 7, 6, 0, 5, 3, 8, 10]
[8, 9, 2, 1, 6, 5, 0, 7, 1, 2]
[9, 9, 5, 10, 5, 3, 7, 0, 4, 7]
[1, 1, 8, 1, 8, 8, 1, 4, 0, 8]
[5, 3, 4, 2, 8, 10, 2, 7, 8, 0]



Brute Force done.
Greedy done.
2-opt LS-Greedy done.
2-opt LS-Random done.
2-opt ILS-Greedy D_Bridge Markovian done.
2-opt ILS-Greedy D_Bridge Random_Walk done.
2-opt ILS-Greedy D_Bridge Restart done.
2-opt ILS-Greedy R_Segment Markovian done.
2-opt ILS-Greedy Swap_Vertices Markovian done.
2-opt ILS-Random D_Bridge Markovian done.
2-opt ILS-Random D_Bridge Random_Walk done.
2-opt ILS-Random D_Bridge Restart done.
2-opt ILS-Random R_Segment Markovian done.
2-opt ILS-Random Swap_Vertices Markovian done.
2-opt ILS R_Initial Markovian done.

SORTED BY COST
+------------------------------------------+--------+--------------+--------------------------------------------+--------------+
| Algorithm                                |   Cost |   Iterations | Path                                       |   Time taken |
+==========================================+========+==============+============================================+==============+
| 2-opt LS-Greedy                          |     30 | 

## Results

As shown in the results obtained from the code above, we gather the following sentiments:

1. Greedy is always the fastest, but often does not have the optimal solution
1. There are a few contenders for what is the "best" solution but it is not clear as each has their drawbacks
1. For a decent approximation, **local search** with an initial **greedy** solution tends to perform well without too many iterations 

**Note that all algorithms using the Restart acceptance criterion is hard capped at 5000 iterations due to iteration limit of 500 and 10 perturbations applied**

## Conclusion

The takeaway from this is that there is no "best" solution and there are simply too many factors specific to the problem that there is no generic good solution. Want a quick solution? Use a greedy algorithm. Want something with a bit more accuracy? Perhaps do a local search. If you need even more accurate results, then run iterative local search until you are happy with the result.